# Scrapping Website Tempo.co 
---
by: __Bakhtiar Amaludin__ | Digital Talent 2018 @ Univ. Gadjah Mada | maziyank@gmail.com | @maziyank
***

Notebook ini adalah hasil dari kegiatan latihan _web scraping_ berita yang ada dalam website tempo.co. _Scraping_ dilakukan melalui halaman indeks berita dengan pembatasan pada tanggal dan kategori berita. Berikut adalah beberapa informasi umum terkait _web scraping_ ini:

- Portal Berita (sumber) : tempo.co (URL: https://www.tempo.co/indeks/)
- Parameter              : Kategori dan Range Tanggal
- Data yang diambil      : Judul, Isi, Tanggal, Author, Editor, Tags, Gambar
- Format _output_        : .json
- Penyimpanan Image      : ./images/

_libraries_ yang digunakan:
- urllib                 : untuk mengambil data dari internet
- StringIO               : untuk menulis file
- BeautifulSoup          : untuk melakukan parsing dan ekstraksi data halaman web
- datetime               : untuk melakukan rutin-rutin yang berhubungan dengan tanggal
- json                   : untuk menyimpan output ke dalam file .json

## Step 1: Inisiasi libraries

In [8]:
import urllib
from bs4 import BeautifulSoup 
from io import StringIO
from datetime import datetime, timedelta
import json
from IPython.display import clear_output

## Step 2: Membuat Fungsi Untuk Mendapatkan URL Berita dari Indeks
---
Sebelum melakukan ekstraksi data berita, terlebih dahulu dilakukan pengumpulan links dari halaman indeks pada website yang akan discrap. Nah, fungsi ini digunakan untuk mendapatkan semua url dari berita yang ada dalam halaman indeks tersebut. Untuk melakukan hal ini terlebih dahulu harus dipahami mengenai struktur dokumen HTML dari website tersebut, sehingga bisa diketahui mana _tag-tag_ yang perlu diambil.

In [3]:
def get_links(start_date, end_date, category):       
    links_berita = []
    for i in range((end_date - start_date).days + 1):
        
        # membuat URL dari halaman indeks dengan format https://www.tempo.co/indeks/{tahun/{bulan}/{tanggal}/{kategori}     
        tgl = start_date + i * timedelta(days=1)
        tgl_str = datetime.strftime(tgl, '%Y/%m/%d')        
        url = 'https://www.tempo.co/indeks/{}/{}'.format(tgl_str, category)      
                
        # mengambil halaman dari url dan menyimpannya dalam variabel string r
        r = urllib.request.urlopen(url).read()     
        
        # ubah variabel r menjadi object BeautifulSoup
        soup = BeautifulSoup(r,"html.parser")

        # cari section khusus yang memuat daftar indeks berita 
        indeks_section = soup.find('section', class_="list list-type-1")

        # pecah section ke dalam daftar card berita
        indeks_berita = indeks_section.find_all('div', class_="card card-type-1")

        # mendapatkan link untuk mengakses single page dari masing-masing berita 
        links_berita.extend([berita.a['href'] for berita in indeks_berita])

    return links_berita                                

## Step 3: Membuat Fungsi Untuk Membaca Berita

Dari _link single page_ berita yang telah diperoleh, langkah berikutnya adalah mengekstrasi data dari halaman yang diperoleh dari _link_ tersebut. Fungsi ini dibuat untuk melakukan tugas tersebut

In [4]:
def getBerita(link):
    # memuat halaman web dari link yang diberikan dan menyimpan ke variabel html
    html = urllib.request.urlopen(link).read()
    
    # mengkonversi variabel html menjadi object BeautifulSoup agar bisa diparsing
    soup = BeautifulSoup(html, "html.parser")

    # mencari bagian yang memuat artikel     
    article = soup.find('article')
    
    # mengambil judul dari artikel     
    judul = article.find("h1", itemprop = "headline").text.strip().encode("utf8").decode()
    
    # mengambil tanggal dari artikel     
    tanggal = article.find("span", itemprop = "datePublished").string
    
    # mengambil isi dari artikel, tetapi masih kasar     
    isi_raw = article.find("div", itemprop = "articleBody").find_all('p')
    
    # membersikan isi dari bagian-bagian yang tidak diperlukan
    isi_clean = '\n'.join([isi.text for isi in isi_raw if 'BACA:' not in isi.text and 'Simak Juga:' not in isi.text])        
    
    # mengambil url gambar dari artikel     
    gambar = article.find("img", itemprop = "image")['src']
    
    # mengambil nama redaksi dari artikel     
    author = article.find("h4", itemprop = "author").string
    
    # mengambil nama editor dari artikel         
    editor = article.find("h4", itemprop = "editor").string
    
    # mengambil keywords dari artikel         
    tags = [t.string for t in article.find_all("a", itemprop = "keywords")]       
    
    # simpan image berita ke folder images
    urllib.request.urlretrieve(gambar, "images" + link[link.rindex('/'):]+gambar[gambar.rindex('.'):])
   
    # mengembalikan nilai dalam bentuk dict, agar mudah diolah menjadi JSON file
    return {"judul": judul, "isi": isi_clean, "tanggal":tanggal, "author":author, "editor":editor, "tags":tags, "gambar": gambar }    

## Step 4: Mulai Scraping 

Proses _web scraping_ dilakukan pada tahap ini menggunakan dua fungsi yang telah dibuat sebelumnya.

In [9]:
# inisiasi variabel untuk menampung hasil scraping
berita = []

# inisiasi parameter yang digunakan sebagai starting point proses scraping
fromDate = datetime(2018,10,22)
toDate   = datetime(2018,10,26)
cat      = 'tekno'

# mendaptkan seluruh links sesuai parameter seberlumnya
links = get_links(fromDate, toDate, cat)

# ekstraksi data dari setiap link yang telah didapat dan menyimpanya ke variabel berita[]
for link in links:
    clear_output()
    berita.append(getBerita(link))
    print('scraping :', link)
      

# menyimpan hasil scraping ke dalam file
with open('tempo_' + cat + '_'+ datetime.strftime(fromDate, '%Y-%m-%d') +'_to_' + datetime.strftime(toDate, '%Y-%m-%d') +'.json', 'w') as outfile:  
    json.dump(berita, outfile)
    print('Scraping Done.. ')



scraping : https://tekno.tempo.co/read/1140023/ftui-buat-rumah-dual-power-pertama-ini-harganya
Scraping Done.. 


In [ ]:
len(berita)